# Optimización Científica del Tamaño de Posición en Estrategias de Trading con Contratos Mixtos

## Resumen

Presentamos un marco computacional y estadístico para la optimización dinámica del tamaño de posición en estrategias de trading discrecional. El enfoque considera un modelo mixto basado en la división de la posición en múltiples contratos con diferentes reglas de salida: dos con toma de beneficios determinística (Take Profit, TP) y uno con perfil estocástico (runner). La metodología integra simulaciones Monte Carlo, teoría de utilidad basada en el criterio de Kelly, y análisis de frontera eficiente con ajustes de varianza para operadores aversos al riesgo. Proponemos extensiones bayesianas y distribuciones no normales para modelar el comportamiento de los beneficios del contrato libre, aportando una herramienta científica reproducible para el análisis riguroso de estrategias personales de gestión de riesgo.

---

## 1. Motivación y marco teórico

La gestión óptima del riesgo en trading ha sido estudiada desde múltiples disciplinas: finanzas matemáticas, teoría de decisiones, control estocástico y teoría de juegos. El problema del sizing óptimo está directamente relacionado con el **Criterio de Kelly** (Kelly, 1956), la **máxima utilidad esperada** (von Neumann & Morgenstern, 1944) y la **teoría de portafolios eficiente** (Markowitz, 1952).

En escenarios donde los resultados de los trades presentan un componente estocástico (como el caso del runner), se requiere una aproximación mixta que combine:

* Evaluación esperada del beneficio por trade.
* Simulación de trayectorias posibles (Monte Carlo).
* Función de utilidad logarítmica ajustada para controlar la varianza y modelar la aversión al riesgo.

---

## 2. Estructura de la estrategia

Se consideran estrategias compuestas por:

* $c_1$: número de contratos con TP fijo.
* $c_r = 3 - c_1$: número de contratos runner.
* Ratio $R$: relación entre el TP y el Stop Loss.
* SL fijo por contrato: $L$.
* El runner tiene beneficio aleatorio $G_r \sim \max(\mathcal{N}(\mu_r, \sigma_r), 0)$.

La ganancia esperada por trade se formula como:

$$
\mathbb{E}[\text{PnL}] = c_1 \cdot [p \cdot R \cdot L - (1 - p) \cdot L] + c_r \cdot \mathbb{E}[G_r]
$$

con $p = \frac{R}{R + 1}$, que asume simetría entre R y probabilidad de éxito basada en riesgo/recompensa.

---

## 3. Función objetivo: utilidad esperada

Se aplica una función de utilidad logarítmica para evaluar la bondad de cada estrategia:

$$
U(\text{PnL}) = \log\left(1 + \frac{\text{PnL}}{\text{Capital}}\right)
$$

Este criterio tiene propiedades deseables:

* Penaliza drawdowns severos (por su concavidad).
* Modela adecuadamente el crecimiento compuesto.
* Se comporta como una versión fraccionada del Kelly Criterion, reduciendo la sensibilidad a errores de estimación.

Cada configuración se simula en múltiples escenarios, acumulando la utilidad esperada y analizando su dispersión (media/desviación estándar).

---

## 4. Visualización y análisis

Para cada estrategia se generan:

* **Trayectorias acumuladas de utilidad logarítmica.**
* **Trayectorias individuales de utilidad logarítmica por trade.**
* **Equity acumulado en unidades monetarias.**
* **Frontera eficiente** riesgo vs retorno logarítmico.

Esta visualización permite evaluar no solo la rentabilidad esperada, sino también la consistencia y robustez frente al riesgo, considerando la aversión subjetiva del operador al drawdown.

---

## 5. Interpretación de variables clave

### Capital base

Se refiere al tamaño total de la cuenta del trader. Este valor se utiliza para:

1. Normalizar la utilidad logarítmica:

$$
\text{Utilidad} = \log\left(1 + \frac{\text{PnL}}{\text{Capital base}}\right)
$$

2. Simular el crecimiento acumulado del capital. A menor capital base, mayor impacto porcentual de cada trade.

**Ejemplo comparativo**:

| Escenario      | PnL por trade | Capital base | log(1 + PnL / capital) | Impacto     |
| -------------- | ------------- | ------------ | ---------------------- | ----------- |
| Trade positivo | \$50          | \$5,000      | 0.00995                | muy pequeño |
| Trade positivo | \$50          | \$100        | 0.405                  | muy grande  |

---

## 6. Crítica y propuestas de mejora

### 6.1 Supuestos limitantes

* La estimación $p = \frac{R}{R + 1}$ es simplificada. Se recomienda calibrarla empíricamente o ajustar dinámicamente vía modelo bayesiano.

* El contrato runner nunca genera pérdidas tras alcanzarse TP1 gracias al movimiento del stop a breakeven. Aunque el código modela esto correctamente mediante truncado inferior, la distribución utilizada (normal) es estadísticamente inadecuada para describir colas largas positivas.

### 6.2 Propuestas de mejora

#### a) Modelado avanzado del runner

Se propone reemplazar la normal truncada por una distribución **log-normal**, **gamma** o **Pareto truncada**, lo que refleja mejor:

* Asimetría positiva.
* Colas largas.
* Ganancias ocasionales extremas.

Ejemplo para log-normal:

$$
G_r \sim \text{LogNormal}(\mu, \sigma), \quad
\mu = \log\left(\frac{\mu_r^2}{\sqrt{\sigma_r^2 + \mu_r^2}}\right), \quad
\sigma = \sqrt{\log\left(1 + \frac{\sigma_r^2}{\mu_r^2}\right)}
$$

#### b) Calibración bayesiana de $p$

Actualizar $p$ tras cada trade usando un modelo bayesiano binomial con prior $\text{Beta}(\alpha, \beta)$:

$$
\hat{p}_{\text{posterior}} = \frac{\alpha + \text{wins}}{\alpha + \beta + \text{total trades}}
$$

#### c) Utilidad ajustada por varianza

$$
U(\text{PnL}) = \log\left(1 + \frac{\text{PnL}}{C}\right) - \lambda \cdot \text{Var}[\text{PnL}]
$$

---

## 7. Referencias académicas clave

* Kelly, J. L. (1956). *A New Interpretation of Information Rate*. Bell System Technical Journal. 35(4), 917–926.
* MacLean, L. C., Thorp, E. O., & Ziemba, W. T. (2011). *The Kelly Capital Growth Investment Criterion: Theory and Practice*. World Scientific.
* Markowitz, H. (1952). *Portfolio Selection*. Journal of Finance, 7(1), 77–91.
* Tversky, A., & Kahneman, D. (1992). *Advances in Prospect Theory: Cumulative Representation of Uncertainty*. Journal of Risk and Uncertainty.
* Garivaltis, A. (2020). *Kelly Criterion for Drawdown Control*. Journal of Risk and Financial Management.
* Filipović, D., & Černý, A. (2021). *Dynamic utility maximization under model uncertainty*. Mathematics and Financial Economics.

---

## 8. Conclusión

Este modelo establece una arquitectura cuantitativa de última generación para la evaluación y optimización de estrategias de trading discrecional. Al incorporar funciones de utilidad ajustadas, modelado probabilístico avanzado y calibración bayesiana, proporciona una herramienta científica con valor práctico y experimental. Su versatilidad lo convierte en un sistema idóneo tanto para traders sistemáticos como para investigadores en gestión del riesgo financiero.




In [7]:

# Total de contratos usados por estrategia: siempre 3
TOTAL_CONTRACTS = 3

# Valores posibles para c1 (contratos con TP fijo) y TP ratio
C1_VALUES = [1, 2, 3]  # posibles cantidades de contratos TP
TP_VALUES = [1.5, 2.0, 2.5, 3.0]  # ratios de take profit

from itertools import product
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import Checkbox, FloatSlider, IntSlider, Dropdown, Layout, VBox, interactive_output
from IPython.display import display, Markdown
import warnings

filtrar_runner_checkbox = Checkbox(
    value=False,
    description='Solo estrategias con runner',
    indent=False
)

def simulate_runner_pnl(mean, std):
    if mean > 0 and std > 0:
        variance = std**2
        mu_log = np.log(mean**2 / np.sqrt(variance + mean**2))
        sigma_log = np.sqrt(np.log(1 + variance / mean**2))
        return np.random.lognormal(mean=mu_log, sigma=sigma_log)
    else:
        return 0.0

def run_analysis(runner_mean, runner_std, n_trades, n_sims, capital_base, contract_value, filtrar_runner_checkbox):
    strategies = []
    SL_VALUE = contract_value

    for c1, rr in product(C1_VALUES, TP_VALUES):
        if c1 > TOTAL_CONTRACTS:
            continue

        c_runner = TOTAL_CONTRACTS - c1
        name = f"{c1}xTP({rr}) + {c_runner}R"

        if filtrar_runner_checkbox and c_runner == 0:
            continue

        config = [(c1, rr)]
        if c_runner > 0:
            config.append((c_runner, 'runner'))

        all_utils = []
        all_raw = []
        for _ in range(n_sims):
            utility_path = []
            raw_path = []
            for _ in range(n_trades):
                trade_pnl = 0
                for contracts, tp in config:
                    if tp == 'runner':
                        pnl = simulate_runner_pnl(runner_mean, runner_std)
                        trade_pnl += contracts * pnl
                    else:
                        p_win = 1 / (1 + 1 / tp)
                        if np.random.rand() < p_win:
                            trade_pnl += contracts * SL_VALUE * tp
                        else:
                            trade_pnl -= contracts * SL_VALUE
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)
                    u = np.log1p(trade_pnl / capital_base)
                utility_path.append(u)
                raw_path.append(u)
            all_utils.append(np.nancumsum(utility_path))
            all_raw.append(raw_path)

        utils = np.array(all_utils)
        raws = np.array(all_raw)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            mean_utility = np.nanmean(utils[:, -1])
            std_utility = np.nanstd(utils[:, -1])

        strategies.append({
            'name': name,
            'c1': c1,
            'TP': rr,
            'c_runner': c_runner,
            'config': config,
            'utils': utils,
            'raws': raws,
            'mean_utility': mean_utility,
            'std_utility': std_utility
        })

    strategies = sorted(strategies, key=lambda x: x['mean_utility'], reverse=True)
    top_names = {s['name'] for s in strategies[:3]}
    best = strategies[0] if strategies else None

    for strat in strategies:
        display(Markdown(f"### 📊 Análisis de estrategia: `{strat['name']}`"))
        fig, axes = plt.subplots(3, 1, figsize=(10, 9), sharex=True)

        for path in strat['utils']:
            axes[0].plot(path, alpha=0.2)
        axes[0].plot(np.nanmean(strat['utils'], axis=0), color='black', linewidth=1.5, label='Media')
        axes[0].set_title(f"{strat['name']} | Log acumulado")
        axes[0].legend()
        axes[0].grid(True)

        for path in strat['raws']:
            axes[1].plot(path, alpha=0.2)
        axes[1].set_title("Trayectoria directa log(pnl/capital)")
        axes[1].grid(True)

        for path in strat['raws']:
            equity_path = np.nancumsum(np.expm1(path) * capital_base)
            axes[2].plot(equity_path, alpha=0.2)
        mean_equity = np.nanmean([np.nancumsum(np.expm1(p) * capital_base) for p in strat['raws']], axis=0)
        axes[2].plot(mean_equity, color='black', linewidth=1.5, label='Media')
        axes[2].legend()
        axes[2].set_title("Equity acumulado ($)")
        final_equity = mean_equity[-1]
        pct_return = (final_equity / capital_base) * 100
        display(Markdown(f"📈 **Equity medio final:** ${final_equity:.2f} ({pct_return:.2f}%)"))
        axes[2].grid(True)

        plt.tight_layout()
        plt.show()

    if strategies:
        df_summary = pd.DataFrame([{
            'Estrategia': s['name'],
            'c1': s['c1'],
            'TP': s['TP'],
            'Runner': s['c_runner'],
            'Kelly Mean': np.round(s['mean_utility'], 2),
            'Kelly Std': np.round(s['std_utility'], 2)
        } for s in strategies])

        display(Markdown(f"### 🧾 Resumen estrategias seleccionadas (Top 10 de {len(df_summary)} combinaciones)"))
        display(df_summary.head(10))

        if best:
            display(Markdown(f"🏆 **Mejor estrategia:** `{best['name']}` con Kelly Mean = {best['mean_utility']:.2f}, Std = {best['std_utility']:.2f}"))

        plt.figure(figsize=(8, 6))
        for _, row in df_summary.iterrows():
            color = 'red' if row['Estrategia'] in top_names else 'blue'
            plt.scatter(row['Kelly Std'], row['Kelly Mean'], color=color)
            plt.text(row['Kelly Std'], row['Kelly Mean'], row['Estrategia'], fontsize=8)

        plt.title("Frontera eficiente: riesgo vs retorno log")
        plt.xlabel("Riesgo (std)")
        plt.ylabel("Retorno esperado (mean)")
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [8]:
# === Widgets ===
runner_mean_slider = FloatSlider(value=12, min=1, max=100, step=1, description='Runner Mean $', layout=Layout(width='400px'))
runner_std_slider = FloatSlider(value=50, min=1, max=100, step=1, description='Runner Std $', layout=Layout(width='400px'))
n_trades_slider = IntSlider(value=50, min=50, max=1000, step=50, description='# Trades', layout=Layout(width='400px'))
n_sims_slider = IntSlider(value=100, min=1, max=500, step=10, description='Monte Carlo Sim', layout=Layout(width='400px'))
contract_dropdown = Dropdown(options={'NQ ($5)': 5, 'ES ($12.5)': 12.5, 'GC ($10)': 10}, value=10, description='Contrato $', layout=Layout(width='400px'))
capital_slider = IntSlider(value=635, min=100, max=10000, step=100, description='Capital base $', layout=Layout(width='400px'))

def update_capital_range(change):
    new_min = change['new'] * 3
    capital_slider.min = new_min
    if capital_slider.value < new_min:
        capital_slider.value = new_min
contract_dropdown.observe(update_capital_range, names='value')

ui = VBox([
    filtrar_runner_checkbox,
    runner_mean_slider,
    runner_std_slider,
    n_trades_slider,
    n_sims_slider,
    contract_dropdown,
    capital_slider
])

out = interactive_output(run_analysis, {
    'runner_mean': runner_mean_slider,
    'runner_std': runner_std_slider,
    'n_trades': n_trades_slider,
    'n_sims': n_sims_slider,
    'contract_value': contract_dropdown,
    'capital_base': capital_slider,
    'filtrar_runner_checkbox': filtrar_runner_checkbox,
})

display(ui, out)


Output()

### 🔍 Interpretación visual de la estrategia `3xTP(3.0) + 0R`

---

![](img/02.png)

#### 🟩 Gráfico superior: **Log acumulado**
- Cada línea representa una simulación independiente bajo la misma configuración.
- Todas las trayectorias son crecientes y presentan **alta regularidad**.
- No hay presencia de saltos o comportamientos atípicos, lo que indica que los trades producen efectos consistentes.
- La línea negra (media) crece de forma sostenida y refleja una **estrategia robusta con crecimiento compuesto estable**.

---

#### 🟦 Gráfico medio: **Trayectoria directa log(PnL/capital)**
- Se observa una alternancia sistemática entre valores positivos y negativos, generando un patrón “en zig-zag”.
- Este comportamiento es típico de estrategias que solo usan TP y SL, sin aleatoriedad introducida por contratos runner.
- La altura del zig-zag corresponde a la magnitud logarítmica de ganancia o pérdida sobre el capital base.

---

#### 🟨 Gráfico inferior: **Equity acumulado ($)**
- Muestra cómo evolucionaría el capital real del trader (en dólares) a lo largo del tiempo.
- Todas las trayectorias están agrupadas, con una dispersión baja, lo que indica **baja volatilidad y alta previsibilidad**.
- No hay equity curves con explosividad ni colapsos, reflejando una estrategia sin sorpresas.

---

### ✅ Conclusión
La estrategia `3xTP(3.0) + 0R` es **puramente determinista**. Al no utilizar contratos runner, ofrece **máxima estabilidad y mínima aleatoriedad**. Es ideal para traders conservadores que priorizan consistencia, control de riesgo y un crecimiento compuesto sostenido, incluso si se renuncia al potencial de ganancias excepcionales.

Esta estrategia es un buen benchmark base contra el que comparar otras configuraciones más arriesgadas.

### 🔍 Interpretación visual de la estrategia `2xTP(3.0) + 1R`

---

![](img/03.png)

#### 🟩 Gráfico superior: **Log acumulado**
- Cada línea representa la trayectoria acumulada de utilidad logarítmica en una de las 101 simulaciones.
- La pendiente general sigue siendo creciente, pero se observa **mayor dispersión** que en estrategias sin runner.
- Esto se debe a la presencia del contrato runner, que tiene un comportamiento aleatorio modelado como una distribución log-normal truncada en cero (breakeven como mínimo).
- Algunas trayectorias muestran crecimiento exponencial leve debido a **outliers positivos** del runner.

---

#### 🟦 Gráfico medio: **Trayectoria directa log(PnL/capital)**
- Aquí se aprecian los valores individuales de cada trade.
- A diferencia de las estrategias sin runner (donde se ven líneas regulares alternando entre ganancias y pérdidas fijas), este gráfico muestra muchos puntos **cercanos a cero** y **picos aislados**.
- Esto refleja que el TP fijo ofrece rentabilidad consistente, pero el runner solo genera beneficios excepcionales de forma esporádica.

---

#### 🟨 Gráfico inferior: **Equity acumulado ($)**
- Muestra la evolución monetaria de la cuenta a lo largo del tiempo.
- La mayoría de curvas siguen un patrón similar, pero algunas presentan saltos abruptos.
- Esos saltos corresponden a trades donde el runner ha alcanzado beneficios significativamente mayores, demostrando el potencial de “cola larga”.
- La línea negra (media) revela que, aunque la estrategia es más volátil, su rendimiento medio sigue siendo elevado.

---

### ✅ Conclusión
La estrategia `2xTP(3.0) + 1R` introduce un elemento de aleatoriedad a través del contrato runner, lo que incrementa la **asimetría** y **dispersión** de resultados. Esta característica hace que sea ideal para traders con **tolerancia moderada al riesgo**, ya que ofrece posibilidad de beneficios excepcionales a costa de una mayor varianza.

Es una estrategia híbrida: combina la estabilidad de los TP fijos con el potencial opcional del runner.


### 🔍 Interpretación visual de la estrategia `3xTP(2.5) + 0R`

Esta imagen contiene tres gráficos, cada uno representando una dimensión crítica de la evolución del capital bajo una estrategia concreta.

---

![](img/01.png)

#### 🟩 Gráfico superior: **Log acumulado**
- Este gráfico muestra la utilidad logarítmica acumulada a lo largo de 50 operaciones simuladas.
- Cada línea de color representa un escenario simulado (una trayectoria posible en Monte Carlo).
- La línea negra es la media de todas las trayectorias.
- La pendiente constante indica una estrategia muy estable, sin sorpresas: gana o pierde siempre en proporciones fijas.
- Como no hay runner (0R), el comportamiento es completamente binario y simétrico.

---

#### 🟦 Gráfico medio: **Trayectoria directa log(PnL/capital)**
- Aquí se visualiza el resultado individual de cada operación, sin acumulación.
- Se observa una clara alternancia entre ganancias (picos positivos) y pérdidas (valles negativos).
- Esta alternancia ordenada indica que las operaciones tienen siempre el mismo impacto relativo sobre el capital.
- Es una firma visual típica de estrategias puramente deterministas y sin componentes aleatorios.

---

#### 🟨 Gráfico inferior: **Equity acumulado en dólares**
- Este gráfico traduce la utilidad logarítmica a términos monetarios (equity).
- Se ve una progresión lineal con ligeras variaciones entre simulaciones.
- No hay saltos grandes ni dispersión extrema, lo que refuerza la idea de estabilidad.
- Esta estrategia es ideal para traders que prefieren consistencia sobre potencial de grandes beneficios.

---

### ✅ Conclusión
La estrategia `3xTP(2.5) + 0R` es un ejemplo de **modelo de gestión cerrado**: no hay aleatoriedad ni variabilidad introducida por contratos runner. Esto permite evaluar su rendimiento con gran precisión. El comportamiento de los tres gráficos confirma su estabilidad y bajo riesgo, a costa de limitar el potencial de beneficio explosivo.

Es una opción sólida para perfiles conservadores que priorizan control de drawdown y previsibilidad en entornos de trading cuantitativo.
